In [ ]:
require 'parser/current'
# opt-in to most recent AST format:
Parser::Builders::Default.emit_lambda              = true

In [ ]:
ruby_ast.type?

In [2]:
require 'rubocop-ast'

false

In [356]:
class Rule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def initialize(rewriter)
    @rewriter = rewriter
  end

  def create_range(begin_pos, end_pos)
    Parser::Source::Range.new(@rewriter.source_buffer, begin_pos, end_pos)
  end
end


class ReplaceRequire < Rule
  def on_send(node)
    if node.method_name == :require
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      @rewriter.replace(node.loc.expression, "import #{req.value}")
    elsif node.method_name == :require_relative
      raise if node.child_nodes.size != 1
      req = node.child_nodes[0]
      raise if !req.str_type?
      if req.value == "lib/baseline_model"
        @rewriter.replace(node.loc.expression, "from lib.baseline_model import BaselineModel")
      else
        raise if req.value != "lib/baseline_model"
      end
    end
  end
end

class ReplaceNew < Rule
  def on_send(node)
    if node.method_name == :new
      range = create_range(node.loc.dot.begin_pos, node.loc.selector.end_pos)
      @rewriter.replace(range, '')
      puts node.receiver
    end
  end
end

class AddParenthesesToMethods < Rule
  def on_send(node)
    if node.arguments.empty? && !(node.loc.expression.source_line.end_with?('()'))
      range = create_range(node.loc.expression.end_pos, node.loc.expression.end_pos)
      @rewriter.replace(range, '()')
      puts node.receiver
    end
  end
end

class ProcessHash < Rule
  def on_hash(node)
    # @rewriter.replace(node.loc.expression, node.loc.expression.source.gsub('=>',':'))
    
    args = []
    node.child_nodes.each do |pair_node|
      raise if !pair_node.pair_type?
      raise if pair_node.child_nodes.size != 2
      raise if !pair_node.child_nodes[0].str_type?
      args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
    end
    @rewriter.replace(node.loc.expression, args.join(', '))
  end
end

:on_hash

In [357]:
# Main method.
def process_file(file, rule_classes=[AddParenthesesToMethods, ReplaceRequire, ReplaceNew, ProcessHash])
  return unless File.exist?(file)
  code = File.read(file)
  frozen_lit = "# frozen_string_literal: true\n\n"
  if code.start_with?(frozen_lit)
    code = code[frozen_lit.size..]
  end
  
  rule_classes.each do |rule_class|
    code = process_rule(rule_class, code)
  end
  code
end

def process_rule(rule_class, code)
  source = RuboCop::AST::ProcessedSource.new(code, 2.7)
  source_buffer = source.buffer
  rewriter = Parser::Source::TreeRewriter.new(source_buffer)
  rule = rule_class.new(rewriter)
  source.ast.each_node { |n| rule.process(n) }
  rewriter.process
end

:process_rule

In [358]:
file_name = 'model/simulationtests/humidity_control.rb'
file_name = 'model/simulationtests/test_port.rb'
file_name = 'model/simulationtests/baseline_sys01.rb'

"model/simulationtests/baseline_sys01.rb"

"model/simulationtests/baseline_sys01.py"

In [360]:
code = process_file(file_name)
code.gsub!('OpenStudio::Model::', 'openstudio.model.')
code.gsub!('Dir.pwd()', 'None')
puts "=" * 80
puts code

(const nil :BaselineModel)
(lvar :model)
(lvar :model)
(lvar :model)
(const nil :BaselineModel)
(const nil :BaselineModel)
(lvar :model)
(lvar :model)
(lvar :model)
(const nil :Dir)
(const nil :Dir)
(const nil :Dir)
(const nil :BaselineModel)
(const nil :BaselineModel)
(const nil :BaselineModel)
import openstudio
from lib.baseline_model import BaselineModel

model = BaselineModel()

# make a 2 story, 100m X 50m, 10 zone core/perimeter building
model.add_geometry(length=100, width=50, num_floors=2, floor_to_floor_height=4, plenum_height=1, perimeter_zone_depth=3)

# add windows at a 40% window-to-wall ratio
model.add_windows(wwr=0.4, offset=1, application_type='Above Floor')

# add ASHRAE System type 01, PTAC, Residential
model.add_hvac(ashrae_sys_num='01')

# add thermostats
model.add_thermostats(heating_setpoint=24, cooling_setpoint=28)

# assign constructions from a local library to the walls/windows/etc. in the model
model.set_constructions()

# set whole building space type; simpli

In [368]:
target_file = File.join(File.dirname(file_name), File.basename(file_name, '.rb') + '.py')
target_file

"model/simulationtests/baseline_sys01.py"

In [370]:
File.write(target_file, code)

941

In [ ]:


class MyRule < Parser::AST::Processor
  include RuboCop::AST::Traversal

  def on_send(node)
    # puts "I found a on_send! #{node}"
    receiver_node, method_name, *arg_nodes = *node

    receiver_node = process(receiver_node) if receiver_node
    node.updated(nil, [
      receiver_node, method_name, *process_all(arg_nodes)
    ])
    if method_name == :require
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      puts req.value
    elsif method_name == :require_relative
      raise if arg_nodes.size != 1
      req = arg_nodes[0]
      raise if !req.str_type?
      raise if req.value != "lib/baseline_model"
      puts req.value
    end
  end
end

rule = MyRule.new
source.ast.each_node { |n| rule.process(n) }

In [233]:
source_code = File.read(file_name)
source = RuboCop::AST::ProcessedSource.from_file(file_name, 2.7)

#<RuboCop::AST::ProcessedSource:0x000056031d413f40 @raw_source="# frozen_string_literal: true\n\nrequire 'openstudio'\nrequire_relative 'lib/baseline_model'\n\nmodel = BaselineModel.new\n\n# make a 2 story, 100m X 50m, 10 zone core/perimeter building\nmodel.add_geometry({ 'length' => 100,\n                     'width' => 50,\n                     'num_floors' => 2,\n                     'floor_to_floor_height' => 4,\n                     'plenum_height' => 1,\n                     'perimeter_zone_depth' => 3 })\n\n# add windows at a 40% window-to-wall ratio\nmodel.add_windows({ 'wwr' => 0.4,\n                    'offset' => 1,\n                    'application_type' => 'Above Floor' })\n\n# add ASHRAE System type 01, PTAC, Residential\nmodel.add_hvac({ 'ashrae_sys_num' => '01' })\n\n# add thermostats\nmodel.add_thermostats({ 'heating_setpoint' => 24,\n                        'cooling_setpoint' => 28 })\n\n# assign constructions from a local library to the walls/windows/etc. in the mode

In [229]:
rule.on_hash

"require 'openstudio'\nrequire_relative 'lib/baseline_model'\n\nmodel = BaselineModel.new\n\n# make a 2 story, 100m X 50m, 10 zone core/perimeter building\nmodel.add_geometry({ 'length' => 100,\n                     'width' => 50,\n                     'num_floors' => 2,\n                     'floor_to_floor_height' => 4,\n                     'plenum_height' => 1,\n                     'perimeter_zone_depth' => 3 })\n\n# add windows at a 40% window-to-wall ratio\nmodel.add_windows({ 'wwr' => 0.4,\n                    'offset' => 1,\n                    'application_type' => 'Above Floor' })\n\n# add ASHRAE System type 01, PTAC, Residential\nmodel.add_hvac({ 'ashrae_sys_num' => '01' })\n\n# add thermostats\nmodel.add_thermostats({ 'heating_setpoint' => 24,\n                        'cooling_setpoint' => 28 })\n\n# assign constructions from a local library to the walls/windows/etc. in the model\nmodel.set_constructions\n\n# set whole building space type; simplified 90.1-2004 Large Office

"require 'openstudio'\nrequire_relative 'lib/baseline_model'\n\nmodel = BaselineModel.new\n\n# make a 2 story, 100m X 50m, 10 zone core/perimeter building\nmodel.add_geometry({ 'length' => 100,\n                     'width' => 50,\n                     'num_floors' => 2,\n                     'floor_to_floor_height' => 4,\n                     'plenum_height' => 1,\n                     'perimeter_zone_depth' => 3 })\n\n# add windows at a 40% window-to-wall ratio\nmodel.add_windows({ 'wwr' => 0.4,\n                    'offset' => 1,\n                    'application_type' => 'Above Floor' })\n\n# add ASHRAE System type 01, PTAC, Residential\nmodel.add_hvac({ 'ashrae_sys_num' => '01' })\n\n# add thermostats\nmodel.add_thermostats({ 'heating_setpoint' => 24,\n                        'cooling_setpoint' => 28 })\n\n# assign constructions from a local library to the walls/windows/etc. in the model\nmodel.set_constructions\n\n# set whole building space type; simplified 90.1-2004 Large Office

In [169]:
# code = "dehumidify_sch = OpenStudio::Model::ScheduleConstant.new(model)"
# source = RuboCop::AST::ProcessedSource.new(code, 2.7)

In [291]:
node = source.ast.each_node.to_a[6]
node

s(:send,
  s(:const, nil, :BaselineModel), :new)

In [294]:
source.ast.each_node.to_a[68]

s(:const, nil, :Dir)

In [297]:
$LIB_METHOD_NAMES = ['add_geometry',
 'add_windows',
 'set_constructions',
 'add_daylighting',
 'add_hvac',
 'set_space_types',
 'add_thermostats',
 'add_design_days',
 'force_year_description',
 'save_openstudio_osm',
 'add_standards',
 'add_schedule',
 'add_water_heater',
 'add_swh_loop',
 'add_swh_end_uses',
 'rename_loop_nodes',
 'rename_air_nodes'
]

["add_geometry", "add_windows", "set_constructions", "add_daylighting", "add_hvac", "set_space_types", "add_thermostats", "add_design_days", "force_year_description", "save_openstudio_osm", "add_standards", "add_schedule", "add_water_heater", "add_swh_loop", "add_swh_end_uses", "rename_loop_nodes", "rename_air_nodes"]

s(:send,
  s(:lvar, :model), :add_windows,
  s(:hash,
    s(:pair,
      s(:str, "wwr"),
      s(:float, 0.4)),
    s(:pair,
      s(:str, "offset"),
      s(:int, 1)),
    s(:pair,
      s(:str, "application_type"),
      s(:str, "Above Floor"))))

In [301]:
node.send_type?

true

In [302]:
node.method_name

:add_geometry

In [307]:
source_code[node.loc.selector.begin_pos...node.loc.selector.end_pos]

"add_geometry"

In [308]:
source_code

"# frozen_string_literal: true\n\nrequire 'openstudio'\nrequire_relative 'lib/baseline_model'\n\nmodel = BaselineModel.new\n\n# make a 2 story, 100m X 50m, 10 zone core/perimeter building\nmodel.add_geometry({ 'length' => 100,\n                     'width' => 50,\n                     'num_floors' => 2,\n                     'floor_to_floor_height' => 4,\n                     'plenum_height' => 1,\n                     'perimeter_zone_depth' => 3 })\n\n# add windows at a 40% window-to-wall ratio\nmodel.add_windows({ 'wwr' => 0.4,\n                    'offset' => 1,\n                    'application_type' => 'Above Floor' })\n\n# add ASHRAE System type 01, PTAC, Residential\nmodel.add_hvac({ 'ashrae_sys_num' => '01' })\n\n# add thermostats\nmodel.add_thermostats({ 'heating_setpoint' => 24,\n                        'cooling_setpoint' => 28 })\n\n# assign constructions from a local library to the walls/windows/etc. in the model\nmodel.set_constructions\n\n# set whole building space type; 

In [309]:
node = source.ast.each_node.to_a[8]
node

s(:send,
  s(:lvar, :model), :add_geometry,
  s(:hash,
    s(:pair,
      s(:str, "length"),
      s(:int, 100)),
    s(:pair,
      s(:str, "width"),
      s(:int, 50)),
    s(:pair,
      s(:str, "num_floors"),
      s(:int, 2)),
    s(:pair,
      s(:str, "floor_to_floor_height"),
      s(:int, 4)),
    s(:pair,
      s(:str, "plenum_height"),
      s(:int, 1)),
    s(:pair,
      s(:str, "perimeter_zone_depth"),
      s(:int, 3))))

In [310]:
node.method_name

:add_geometry

In [331]:
node = source.ast.each_node.select { |node|
  next if !node.send_type?
  next if ! $LIB_METHOD_NAMES.include?(node.method_name.to_s)
  node
}[1]

s(:send,
  s(:lvar, :model), :add_windows,
  s(:hash,
    s(:pair,
      s(:str, "wwr"),
      s(:float, 0.4)),
    s(:pair,
      s(:str, "offset"),
      s(:int, 1)),
    s(:pair,
      s(:str, "application_type"),
      s(:str, "Above Floor"))))

In [332]:
hash_node = node.arguments[0]
hash_node

s(:hash,
  s(:pair,
    s(:str, "wwr"),
    s(:float, 0.4)),
  s(:pair,
    s(:str, "offset"),
    s(:int, 1)),
  s(:pair,
    s(:str, "application_type"),
    s(:str, "Above Floor")))

In [355]:
args = []
hash_node.child_nodes.each do |pair_node|
  raise if !pair_node.pair_type?
  raise if pair_node.child_nodes.size != 2
  raise if !pair_node.child_nodes[0].str_type?
  args << "#{pair_node.child_nodes[0].value}=#{pair_node.child_nodes[1].loc.expression.source}"
end
puts args.join(', ')

wwr=0.4, offset=1, application_type='Above Floor'


In [335]:
pair_node = hash_node.child_nodes[0]
pair_node

s(:pair,
  s(:str, "wwr"),
  s(:float, 0.4))

In [336]:
pair_node.child_nodes.size

2

In [350]:
str_node = pair_node.child_nodes[0]

s(:str, "wwr")

In [353]:
str_node.value

"wwr"

In [345]:
"#{pair_node.child_nodes[0].loc.expression.source}=#{pair_node.child_nodes[1].loc.expression.source}"

"'wwr'=0.4"

In [315]:
child_node.child_nodes[0].value

"length"

In [316]:
child_node.child_nodes[1].value

100

In [319]:
str_node = child_node.child_nodes[0]
str_node

s(:str, "length")

In [275]:
node.loc.expression.source.gsub('=>',':')

"{ 'length' : 100,\n                     'width' : 50,\n                     'num_floors' : 2,\n                     'floor_to_floor_height' : 4,\n                     'plenum_height' : 1,\n                     'perimeter_zone_depth' : 3 }"

In [265]:
print(source_code[node.loc.expression.begin_pos...node.loc.expression.end_pos])
node.loc.expression.source.gsub('=>',':')

{ 'length' => 100,
                     'width' => 50,
                     'num_floors' => 2,
                     'floor_to_floor_height' => 4,
                     'plenum_height' => 1,
                     'perimeter_zone_depth' => 3 }

In [263]:
node.loc.expression.begin_pos

199

In [213]:
node = source.ast.each_node.to_a[24]
node

s(:if,
  s(:send,
    s(:lvar, :zones), :nil?),
  s(:send, nil, :puts,
    s(:str, "hello")), nil)

In [214]:
node.if?

true

In [217]:
n = node.child_nodes[0]
n

s(:send,
  s(:lvar, :zones), :nil?)

In [218]:
n.method_name

:nil?

In [157]:
source_code[113...116]

"new"

In [179]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

model = BaselineModel.new()


true

In [180]:
source_code[node.loc.dot.begin_pos...node.loc.expression.end_pos]

".new()"

In [175]:
node.loc.expression.end

#<Parser::Source::Range model/simulationtests/humidity_control.rb 144...144>

In [133]:
source.ast.each_node.to_a[8]

s(:send,
  s(:lvar, :model), :add_geometry,
  s(:hash,
    s(:pair,
      s(:str, "length"),
      s(:int, 100)),
    s(:pair,
      s(:str, "width"),
      s(:int, 50)),
    s(:pair,
      s(:str, "num_floors"),
      s(:int, 2)),
    s(:pair,
      s(:str, "floor_to_floor_height"),
      s(:int, 4)),
    s(:pair,
      s(:str, "plenum_height"),
      s(:int, 1)),
    s(:pair,
      s(:str, "perimeter_zone_depth"),
      s(:int, 3))))

In [154]:
node.loc

#<Parser::Source::Map::Send:0x000056031c399840 @dot=#<Parser::Source::Range model/simulationtests/humidity_control.rb 112...113>, @selector=#<Parser::Source::Range model/simulationtests/humidity_control.rb 113...116>, @end=#<Parser::Source::Range model/simulationtests/humidity_control.rb 117...118>, @begin=#<Parser::Source::Range model/simulationtests/humidity_control.rb 116...117>, @expression=#<Parser::Source::Range model/simulationtests/humidity_control.rb 99...118>, @node=s(:send,
  s(:const, nil, :BaselineModel), :new)>

In [155]:
node.loc.expression.source_line.end_with?'()'

true

In [102]:
node.arguments.empty?

true

In [57]:
"BaselineModel.new".size

17

In [104]:
node.receiver

s(:const, nil, :BaselineModel)

In [62]:
source_code[113...116]

"new"

In [83]:
node.child_nodes

[s(:const, nil, :BaselineModel)]

116

In [75]:
node = source.ast.each_node.to_a[76]
node

s(:lvasgn, :dehumidify_sch,
  s(:send,
    s(:const,
      s(:const,
        s(:const, nil, :OpenStudio), :Model), :ScheduleConstant), :new,
    s(:lvar, :model)))

In [184]:
node = source.ast.each_node.to_a[77]
node

s(:lvar, :zones)

In [188]:
node = source.ast.each_node.to_a[98]

s(:send,
  s(:send,
    s(:lvar, :zone), :airLoopHVAC), :get)

In [190]:
s = node.loc.expression.source_line
puts s
s.end_with?'()'

  air_system = zone.airLoopHVAC.get


false

In [92]:
node.arguments

[s(:lvar, :model)]

In [78]:
node.method_name

:new

In [80]:
node.sibling_index

1

In [79]:
x = node.receiver
x

s(:const,
  s(:const,
    s(:const, nil, :OpenStudio), :Model), :ScheduleConstant)

In [108]:
x.const_type?

true

In [109]:
x.module_name?

true

In [111]:
x.class_name?

true

In [114]:
x

s(:const,
  s(:const,
    s(:const, nil, :OpenStudio), :Model), :ScheduleConstant)

In [29]:
x.const_type?

true

In [36]:
x.children[0].children[0]

s(:const, nil, :OpenStudio)

In [41]:
x.children[0]

s(:const,
  s(:const, nil, :OpenStudio), :Model)

In [ ]:
node.child_nodes

In [ ]:
x.const_name

In [ ]:
receiver_node, method_name, *arg_nodes = *node

In [ ]:
node.method_name

In [ ]:
node.child_nodes

In [ ]:
req = arg_nodes[0]

In [ ]:
req.str_type?

In [ ]:
req.value .

In [ ]:
node.loc.selector

In [ ]:
"require 'openstudio'".size

In [ ]:
"require".size

In [ ]:
node.loc.selector

In [ ]:
node.loc.expression.begin_pos

In [ ]:
node.loc.expression.end_pos

In [ ]:
node.loc